In [8]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import Statevector
import numpy as np
from qiskit import Aer, QuantumCircuit, execute
import pandas as pd
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics.pairwise import rbf_kernel

# CLASSIC FUNCTIONS AND FIDELITY_SV

In [9]:
# funzione per fit dentro fidelity_per_site
def exp_func(x, a, b):
    return a * np.exp(b*x)

# per kernel classico
def rbf_func(x1,x2, circuit, gamma = 0.05):
    #assert len(x1)==len(x2)
    distance_squared = np.sum((x1 - x2) ** 2)
    kernel_value = np.exp(-gamma * distance_squared)
    return kernel_value

# fidelity tramite StateVector
def fidelity_SV(x1,x2, circuit):
    encoded_x_i = circuit.assign_parameters(x1)
    encoded_x_j = circuit.assign_parameters(x2)
    statevector_i = Statevector.from_instruction(encoded_x_i)
    statevector_j = Statevector.from_instruction(encoded_x_j)
    kernel_value = abs(statevector_j.conjugate().data @ statevector_i.data)**2
    return kernel_value

# FIDELITIES KERNELS

In [3]:
backend = Aer.get_backend('qasm_simulator')

# fidelity tramite QuasmSimulator
def fidelity(x1, x2,  circuit, shots=1000):
    assert len(x1) == len(x2)
    n = len(x1)
    encoded_x_i = circuit.assign_parameters(x1)
    encoded_x_j = circuit.assign_parameters(x2)
    qc = QuantumCircuit(N, N)
    qc.append(encoded_x_i, range(N))
    qc.append(encoded_x_j.inverse(), range(N)) 
    qc.measure(range(N), range(N))
    counts = execute(qc, backend, shots=shots).result().get_counts()

    # calcolo della sovrapposzione tramite frequenza di outcome |0>
    kernel_value = np.sqrt(counts.get('0' * N, 0) / shots)
    return kernel_value

# fidelity per sito (lambda)
def fidelity_per_site(x1, x2,  circuit, n_start=1, reps=4, shots=2000):
    assert len(x1) == len(x2)
    n_meas=n_start
    encoded_x_i = circuit.assign_parameters(x1)
    encoded_x_j = circuit.assign_parameters(x2)
    p_0=[]
    Ns_meas = []

    for _ in range(reps):
        n_meas += 1
        Ns_meas.append(n_meas)

        # varia il numero di qubit misurati ogni volta
        qc = QuantumCircuit(N, n_meas)
        qc.append(encoded_x_i, range(N))
        qc.append(encoded_x_j.inverse(), range(N)) 
        qc.measure(range(n_meas), range(n_meas))

        # misura della sovrapposzione, limitata nel numero di qubit
        counts = execute(qc, backend, shots=shots).result().get_counts()
        p_0.append(counts.get('0' * n_meas, 0) / shots)
    p_0 = np.array(p_0)
    Ns_meas = np.array(Ns_meas)

    # fit per determinare lambda(n) n: numero di bit, ed estrapolare Lambda(N) N grande
    popt, pcov = curve_fit(exp_func, Ns_meas, p_0)
    kernel_value = exp_func(N, *popt)**(1/(2*N))
    return kernel_value

# NOISY FIDELITY KERNELS

In [13]:
from qiskit_aer.primitives import Sampler
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.result import  ProbDistribution


# metodi di error mitigation, non eseguiti per ragioni di tempo computazionale troppo elevato
"""from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options
service=QiskitRuntimeService()
options = Options()
options.resilience_level = 1
options.optimization_level = 3
backend = service.backend("ibmq_qasm_simulator")
sampler = Sampler(options=options, backend = backend)"""

# errore di decoerenza nei gate CNOT, con probabilità variabile
noise_model = NoiseModel()
cx_depolarizing_prob = 0.1
noise_model.add_all_qubit_quantum_error(depolarizing_error(cx_depolarizing_prob, 2), ["cx"])
sampler = Sampler(backend_options={"noise_model": noise_model})

#--------------------------------------------------------------------
# fidelity col sampler, utilizzata per ragioni di migliore compatibilità nel caso di rumore

# equivalente a quella con QuasmSimulator
def fidelity_sampler(x1, x2,  circuit, shots=1000):
    assert len(x1) == len(x2)
    encoded_x_i = circuit.assign_parameters(x1)
    encoded_x_j = circuit.assign_parameters(x2)
    qc = QuantumCircuit(N, N)
    qc.append(encoded_x_i, range(N))
    qc.append(encoded_x_j.inverse(), range(N)) 
    qc.measure(range(N), range(N))
    job=sampler.run(qc, shots=shots)
    result = job.result()
    quasi_dist=result.quasi_dists[0]
    if 0 in quasi_dist:
        kernel_value =np.sqrt(abs(quasi_dist[0]))
    else:
        kernel_value=0
    return kernel_value

# alternativa equivalente suggerita nella documentazione Qiskit
def fidelity_sampler_bis(x1, x2, circuit, shots=1000):
    qc1 = circuit.assign_parameters(x1)
    qc2 = circuit.assign_parameters(x2)
    qc1.measure_all()
    qc2.measure_all()
    job1 = sampler.run(qc1, shots= shots)
    job2 = sampler.run(qc2, shots= shots)
    quasis1 = job1.result().quasi_dists[0]
    quasis2 = job2.result().quasi_dists[0]
    dist1 = quasis1.nearest_probability_distribution()
    dist2 = quasis2.nearest_probability_distribution()
    result=0
    for bitstring in dist1 | dist2:
        prob1 = dist1.get(bitstring, 0)
        prob2 = dist2.get(bitstring, 0)
        result += np.sqrt(prob1*prob2)
    return result**2

# CLASS MODEL

In [11]:
class Model():

    def __init__(self,N_spins):
        self.N_spins = N_spins
        self.circuit = RealAmplitudes(num_qubits=N_spins, reps=3)
        self.data = None
        self.N_dataset = None
        self.J_alldata = None
        self.parameters_alldata = None
        self.trainset_size = None
        self.data_train = None
        self.parameters_train = None
        self.J_train = None
        self.distance = None
        self.k_matrix = None
        self.SVM = SVC(kernel='precomputed')
        self.y = None
        self.dual_coeff = None
        self.support_vectors = None
        self.accuracy = None
        self.J_test = None
        self.test_data = None
        self.test_parameters = None
        self.test_size = 200
        self.smoothed_distance = None
        self.smoothed_J_test = None
        

    def initialize(self, data_file = 'dataset/dataset_nuovo_600.csv', trainset_size=100, training_set= 'delta'):
        self.trainset_size = trainset_size
        data = pd.read_csv(data_file)
        self.data = np.array(data)
        self.J_alldata = self.data[:,0]
        self.parameters_alldata = self.data[:,3:]
        self.N_dataset = len(self.data)
        if training_set=='delta':
            indici = np.random.choice(self.N_dataset, size=trainset_size, replace=False); indici.sort()
        elif training_set=='sigma_1':
            indici1 = np.arange(201, 251)
            indici2 = np.arange(350, 400)
            indici = np.concatenate((indici1,indici2))
        elif training_set=='sigma_2':
            indici1 = np.arange(151, 201)
            indici2 = np.arange(400, 450)
            indici = np.concatenate((indici1,indici2))
        elif training_set=='sigma_3':
            indici1 = np.range(219,261)
            indici2 = np.range(540,583)
            indici = np.concatenate((indici1,indici2))
        else:
            print("Intervallo per il trainset non riconosciuto")
        self.train_data = self.data[indici, :]
        self.train_parameters = self.train_data[:,3:]
        self.J_train = self.train_data[:,0]
        self.y = np.where(self.J_train.astype(float)<1,-1,1)
        self.test_data = self.data[200:400, :]
        self.J_test = self.test_data[:,0]
        self.test_parameters = self.test_data[:,3:]

#--------------------------------------------------------------
    # calcolo della matrice dei kernel, con funzione di kernel variabile
    def kernel_matrix_construct(self, kernel_func, save = True, save_file='kernel_matrix'):
        k_matr = np.zeros(shape=(self.trainset_size,self.trainset_size))
        for i in range(self.trainset_size):
            print(f'Calcolo kernel, riga numero={i}', end='\r')
            x1 = self.train_parameters[i,:]
            # si calcola solamente la matrice triangolare inferiore: il risultato deve essere simmetrico
            for j in range(i):
                x2 = self.train_parameters[j,:]
                k_matr[i,j] = kernel_func(x1,x2,circuit = self.circuit)
        
        # simmetrizzazione della matrice ed aggiunta degli 1 sulla diagonale
        k_matr_simmetrized = k_matr + k_matr.transpose() + np.diag(np.ones(self.trainset_size))
        self.k_matrix = k_matr_simmetrized
        if save:
            np.savetxt(save_file + '.txt', k_matr_simmetrized)
            np.save(save_file + '.npy', k_matr_simmetrized)

    def kernel_matrix_plot(self):
        plt.imshow(self.k_matrix, origin='lower')
        plt.title("Kernel matrix")
        xx = np.linspace(0,99,5).astype(int)
        jj = np.round(self.J_train[xx],2)
        plt.xticks(xx,jj); plt.yticks(xx,jj)
        plt.colorbar()
        plt.show()

    # calcolo della SVM, accuratezza ed indici dei vettori di supporto
    def driver_SVM(self):
        self.SVM.fit(self.k_matrix, self.y)
        prediction=self.SVM.predict(self.k_matrix)
        correct = np.sum(self.y == prediction)
        self.accuracy = correct / len(self.y)

        # coefficienti nel calcolo delle distante relativi ai vettori di support
        self.dual_coeff = self.SVM.dual_coef_

        # indici dei vettori di supporto (nel training set) che ritronano coefficienti non nulli 
        self.support_vectors = self.SVM.support_
        print(f'Accuracy of the SVM = {self.accuracy}')
        print(f'Offset parameter b= {self.SVM.intercept_}')

    # calcolo di d(J)
    def distance_calc(self, kernel_func, save=True, save_file = 'distances'):
        self.distance = np.zeros(self.test_size)
        print("---------------------------")
        # la distanza si può calcolare per J qualsiasi
        for i in range(self.test_size):
            print(f'Calcolo distance, riga numero={i}',end='\r')
            x1 = self.test_parameters[i,:]

            # ma all'interno della formula, il secondo indice del kernel corrisponde ai vettori di supporto
            for num_j,j in enumerate(self.support_vectors):
                x2=self.train_parameters[j,:]
                self.distance[i] += self.dual_coeff[0,num_j]*kernel_func(x1,x2,circuit=self.circuit)
        self.distance += self.SVM.intercept_
        print("---------------------------")
        if save:
            np.save(save_file, self.distance)
        
        # per rendere il risultato più pulito, la funzione distanza viene "levigata" tramite coarse graining
        self.smoothed_distance = np.convolve(self.distance, np.ones(20)/20, mode='valid')
        self.smoothed_J_test= np.convolve(self.J_test, np.ones(20)/20, mode='valid')
        if save:
            np.save('sm_' + save_file + 'npy', self.smoothed_distance)

        # e J_c calcolata come l'intersizione delle distanze con 0
        print(f'J_phase_transition = {self.J_test[abs(self.distance).argmin()]}',
              f'dist_min = {abs(self.distance[abs(self.distance).argmin()])}')
        print(f'smoothed_J_phase_transition = {self.smoothed_J_test[abs(self.smoothed_distance).argmin()]}',
              f'smoothed_dist_min = {abs(self.smoothed_distance[abs(self.smoothed_distance).argmin()])}')

    def distance_plot(self):
        plt.plot(self.J_test,self.distance, label = 'Distance')
        plt.plot((self.smoothed_J_test),self.smoothed_distance, label='Coarse grained distance')
        plt.legend()
        plt.show()


# 10 SPIN CHAIN

# Rbf kernel

In [ ]:
# kernel classico su 10 spin
N=10
chain_10_rbf = Model(N)
chain_10_rbf.initialize()
chain_10_rbf.k_matrix = rbf_kernel(chain_10_rbf.train_parameters,gamma=0.05)
chain_10_rbf.kernel_matrix_plot()
chain_10_rbf.driver_SVM()
chain_10_rbf.distance_calc(rbf_func, save=False)
chain_10_rbf.distance_plot()

# Fidelities w noise

In [ ]:
# kernel fidelity + rumore (impsotabile su 0)
# il caso di fidelity per sito è stato eseguito analogamente, 
# basta variare le funzioni in ingresso a kernel_matrix_construct e distance_calc

N=10
chain_10_fidelity = Model(N)
chain_10_fidelity.initialize()
chain_10_fidelity.kernel_matrix_construct(fidelity, save_file='fidelity_simul')
chain_10_fidelity.kernel_matrix_plot()
chain_10_fidelity.driver_SVM()
chain_10_fidelity.distance_calc(fidelity, save_file='distances_simul')
chain_10_fidelity.distance_plot()

In [ ]:
N=10
chain_10_fidelity_SV = Model(N)
chain_10_fidelity_SV.initialize()
chain_10_fidelity_SV.kernel_matrix_construct(fidelity_sampler, save_file='ourfidelity_sampler_10s_1000s_k')
chain_10_fidelity_SV.kernel_matrix_plot()
chain_10_fidelity_SV.driver_SVM()
chain_10_fidelity_SV.distance_calc(fidelity_sampler, save_file='ourfidelity_sampler_10s_1000s_d')
chain_10_fidelity_SV.distance_plot()

# 5 SPIN CHAIN

# Rbf kernel

In [ ]:
N=5
gamma=2.5
prova0 = Model(N)
prova0.initialize(data_file='dataset/dataset_5_spin.csv', training_set='delta')
prova0.kernel_matrix_construct(lambda x1, x2, circuit: rbf_func(x1, x2, circuit=None, gamma=gamma))
prova0.kernel_matrix_plot()
prova0.driver_SVM()
prova0.distance_calc(lambda x1, x2, circuit: rbf_func(x1, x2, circuit=None, gamma=gamma), save=False)
prova0.distance_plot()

# Fidelities

In [ ]:
N=5
gamma=2
prova0 = Model(N)
prova0.initialize(data_file='dataset/dataset_5_spin.csv', training_set='delta')
prova0.kernel_matrix_construct(fidelity)
prova0.kernel_matrix_plot()
prova0.driver_SVM()
prova0.distance_calc(fidelity)
prova0.distance_plot()